<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymorphy2[fast]

In [0]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Check out [this issue](https://github.com/hanxiao/bert-as-service/issues/380) and "make sure Colab is using Tensorflow 1.x, because bert-serving-start doesn't currently work with TF 2.1 and nohup hides the output of the command failing"

In [0]:
import tensorflow as tf
print (tf.__version__)

1.15.0


In [0]:
!pip install -U bert-serving-server[http]
!pip install bert-serving-client  # client, independent of `bert-serving-server`

     |████████████████████████████████| 71kB 4.4MB/s 
     |████████████████████████████████| 1.1MB 19.9MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=04504f7de505382fbe26311b73c34e620decce0a0be76b01de41eed36a1adbf3
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for flask-compress: filename=Flask_Compress-1.4.0-cp36-none-any.whl size=3712 sha256=fc00a9bd98b65e2f638d66a67b891b2946d76caff7a3aaeefdfd6b2110e5db3e
  Stored in directory: /root/.cache/pip/wheels/96/32/88/a1f6d9dd3c29570ab3a8acc0d556b3b20abcf3c623c868ce0a
Successfully built GPUtil flask-compress
  Found existing installation: pyzmq 17.0.0
    Uninstalling pyzmq-17.0.0:
      Successfully uninstalled pyzmq-17.0.0


In [0]:
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip /content/multi_cased_L-12_H-768_A-12.zip

--2020-02-21 09:32:02--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 2a00:1450:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M  68.2MB/s    in 9.4s    

2020-02-21 09:32:12 (67.2 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

Archive:  /content/multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_

In [0]:
!nohup bert-serving-start -model_dir=./multi_cased_L-12_H-768_A-12 > out.file 2>&1 &

In [0]:
from bert_serving.client import BertClient
bc = BertClient()
encoded_test = bc.encode(['First do it', 'then do it right', 'then do it better'])
encoded_test

array([[ 0.49155793,  0.08796163,  0.08263294, ...,  1.0980438 ,
         0.411266  , -0.2539694 ],
       [-0.07413446, -0.22783582, -0.08978209, ...,  1.5093104 ,
         1.3512006 , -0.03158369],
       [-0.2657499 ,  0.1913579 , -0.35613576, ...,  1.3962182 ,
         1.4187475 ,  0.12651818]], dtype=float32)

In [0]:
encoded_test = bc.encode(['Вася любит Машу'])
encoded_test

array([[-1.23986453e-01, -8.49756449e-02,  5.41371465e-01,
         6.32736683e-01, -3.28210145e-01, -1.50357351e-01,
        -1.03384745e+00,  5.56825213e-02,  5.22987098e-02,
         8.24393988e-01,  6.84741437e-01, -8.14457476e-01,
        -2.43044689e-01, -4.22898903e-02, -1.46446094e-01,
        -6.54376805e-01, -1.40329540e-01,  2.79075503e-01,
        -4.67326900e-04, -2.01118737e-01,  1.08960402e+00,
         3.28487396e-01, -9.49426234e-01, -1.79649368e-01,
        -2.15582401e-01, -2.89596677e-01,  8.33332539e-02,
        -5.88268280e-01,  4.92330015e-01,  8.24053884e-01,
         4.18344617e-01,  2.52877995e-02,  1.22798598e+00,
        -2.62517303e-01, -2.16584533e-01,  8.61240923e-01,
        -3.81806254e-01, -1.61617100e-01, -7.70443439e-01,
         5.74837863e-01, -6.20551109e-01,  7.99862981e-01,
         6.99325502e-02,  1.63031995e-01,  7.39420801e-02,
        -8.39173079e-01,  2.09508777e-01,  5.40831923e-01,
        -1.01098204e+00, -1.07717299e+00,  5.71467102e-0

In [0]:
for vec in encoded_test:
  print (len(vec))


In [0]:
# arg0, arg1, verb = 'маша петя любить'.split
def a0_a1_sentence_maker(verb, arg0, arg1, arg0_case='nomn', arg1_case='accs', cap=True):

  '''Генерирует простые предложения вида 'arg0 любит arg1'.
  Параметры:
  verb              глагол в любой форме;
  arg0_case='nomn'  тег падежа подлежащего;
  arg1_case='accs'  тег падежа дополнения;
  arg0='Маша'       подлежащее;
  arg1='Петя'       дополнение
  cap=True        капитализация аргументов-имен собственных.
  Допустимые теги падежей лежат тут: http://opencorpora.org/dict.php?act=gram.'''

  inflected_arg0 = morph.parse(arg0)[0].inflect({'sing', 'nomn'}).word
  inflected_verb = morph.parse(verb)[0].inflect({'sing', '3per', 'pres', 'indc'}).word
  inflected_arg1 = morph.parse(arg1)[0].inflect({'sing', 'accs'}).word

  if (cap == True) and ('Name' in morph.parse(arg1)[0].tag):
    inflected_arg1 = inflected_arg1.capitalize()
  if (cap == True) and ('Name' in morph.parse(arg0)[0].tag):
    inflected_arg0 = inflected_arg0.capitalize()

  sentence = '{} {} {}'.format(inflected_arg0, inflected_verb, inflected_arg1)
  return sentence

a0_a1_sentence_maker('любить', 'Маша', 'Петя')
# a0_a1_sentence_maker('любить', 'Петя', 'Маша')


  

'Маша любит Петю'

In [0]:
# bc.server_status